[![preview notebook](https://img.shields.io/static/v1?label=render%20on&logo=github&color=87ce3e&message=GitHub)](https://github.com/open-atmos/devops_tests/blob/main/test_files/template.ipynb)
[![launch on mybinder.org](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/open-atmos/devops_tests.git/main?urlpath=lab/tree/test_files/template.ipynb)
[![launch on Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/open-atmos/devops_tests/blob/main/test_files/template.ipynb)

In [1]:
import sys

from tests.unit_tests.backends.test_oxidation import formulae

if 'google.colab' in sys.modules:
    !pip --quiet install open-atmos-jupyter-utils
    from open_atmos_jupyter_utils import pip_install_on_colab
    pip_install_on_colab('PySDM-examples')

In [2]:
import numpy as np

In [10]:
radii = np.asarray([0.005, 0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.20]) * si.cm
a = radii[0]
f = 0.75
c1 = 1.63
rho_v = 0.804 #change
rho = 1.225 #change
D = 0.26 #change

In [63]:
F = 0.75 #change
Re = 3000 #change
eta = 1.48*10**(-5) #change

In [64]:
def rates_of_evaporation(radius):
    return -3/(radius**2)*D*rho_v*(1 + F*np.sqrt(Re)/np.sqrt(4*np.pi*rho*D/eta))

In [69]:
def relaxation_time(radius):
    return (c1*rates_of_evaporation(radius))**(-1)

In [70]:
print(relaxation_time(a))

-2.3002021125729345e-07


In [28]:
import pandas
from PySDM.physics import in_unit, si
from PySDM import Formulae

formulae = Formulae(
    terminal_velocity='RogersYau',
    drop_growth='Mason1951',
    diffusion_thermics='Neglect',
    saturation_vapour_pressure='AugustRocheMagnus',
    ventilation='Froessling1938',
    particle_shape_and_density='LiquidSpheres',
)
const = formulae.constants
any_non_zero_value = 44.
temperature = const.T0 + 10 * si.K
pressure = const.p_STP
pvs = formulae.saturation_vapour_pressure.pvs_water(temperature)
# Re = formulae.particle_shape_and_density.reynolds_number(radius, velocity_wrt_air, dynamic_viscosity, density)
# Sc = 
# f = formulae.ventilation.ventilation_coefficient(sqrt_re_times_cbrt_sc=Re**(1/2) * Sc**(1/3))
f = .75

r_dr_dt = formulae.drop_growth.r_dr_dt(
    RH_eq=1,
    T=temperature,
    RH=0,
    lv=0,
    pvs=pvs,
    D=formulae.diffusion_thermics.D(T=temperature, p=pressure),
    K=any_non_zero_value,
    ventilation_factor=f
)
adjustment_time = (-3 / radii**2 * r_dr_dt * c1)**-1

v_term = formulae.terminal_velocity.v_term(radii)

data = pandas.DataFrame({
    'radius [cm]': in_unit(radii, si.cm),
    'adjustment time [s]': adjustment_time,
    'terminal velocity [m/s]': v_term,
    'd [m]': v_term * adjustment_time
})

data

,radius [cm],adjustment time [s],terminal velocity [m/s],d [m]
0,0.005,3.214946,0.400000,1.285978
1,0.010,12.859783,0.800000,10.287826
2,0.025,80.373641,2.000000,160.747282
3,0.050,321.494564,4.000000,1285.978258
4,0.075,723.362770,5.504612,3981.831170
5,0.100,1285.978258,6.356178,8173.906836
6,0.150,2893.451080,7.784697,22524.638573
7,0.200,5143.913032,8.988993,46238.599620


In [25]:
a = np.zeros((1,2,3))
print(a.ctypes.data, np.asarray(a).ctypes.data, np.array(a).ctypes.data)

105553144971456 105553144971456 105553144971264
